In [1]:
import pandas as pd
import numpy as np

In [2]:
data_por = pd.read_csv("student-por.csv", sep = ';')

In [3]:
pd.set_option('display.max_columns', None)

### remove G2 & G3 = 0 

In [4]:
data_por_2 = data_por.copy()

In [5]:
result_0 = data_por_2[data_por_2["G3"]==0].index
data_por_2 = data_por_2.drop(index = result_0)
data_por_2 = data_por_2.reset_index()
data_por_2 = data_por_2.drop(columns = ["index"])

In [6]:
data_por_2

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,MS,F,19,R,GT3,T,2,3,services,other,course,mother,1,3,1,no,no,no,yes,no,yes,yes,no,5,4,2,1,2,5,4,10,11,10
630,MS,F,18,U,LE3,T,3,1,teacher,services,course,mother,1,2,0,no,yes,no,no,yes,yes,yes,no,4,3,4,1,1,1,4,15,15,16
631,MS,F,18,U,GT3,T,1,1,other,other,course,mother,2,2,0,no,no,no,yes,yes,yes,no,no,1,1,1,1,1,5,6,11,12,9
632,MS,M,17,U,LE3,T,3,1,services,services,course,mother,2,1,0,no,no,no,no,no,yes,yes,no,2,4,5,3,4,2,6,10,10,10


### Seperate data into train and test

In [7]:
import random as rd

In [8]:
rd.seed(5319)
data_dict = data_por_2.to_dict("index")
data_dict_list = list(data_dict)
test_index = rd.sample(data_dict_list, k = 100)

In [9]:
def train_test_seperate(df_dict, index_list, test_list_index):
    data_train = {}
    data_test = {}
    for i in range(len(index_list)):
        if index_list[i] in test_list_index:
            data_test[i] = df_dict[i]
        else:
            data_train[i] = df_dict[i]
    data_train = pd.DataFrame.from_dict(data_train, orient = 'index')
    data_test = pd.DataFrame.from_dict(data_test, orient = 'index')
    return data_train, data_test

In [10]:
data_train, data_test = train_test_seperate(data_dict, data_dict_list, test_index)

### categorical transform

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
def categorical_transform(df):
    df = df.reset_index()
    df = df.drop(columns = ["index"])
    column_name = list(df.columns)
    transform_dic = {}
    for i in range(len(column_name)):
        labelencoder = LabelEncoder()
        if type(df[column_name[i]][1]) == str:
            df[column_name[i]] = labelencoder.fit_transform(df[column_name[i]])
            transform_dic[column_name[i]] = list(labelencoder.classes_)
    return df, transform_dic

In [13]:
data_train_2 = data_train.copy()
data_train_2, categorical_dic = categorical_transform(data_train_2)

In [14]:
data_train_2.head()

,Dalc,traveltime,Medu,internet,Fedu,paid,reason,sex,guardian,Walc,freetime,G2,activities,romantic,G3,higher,famsup,failures,absences,nursery,G1,school,health,Pstatus,goout,age,famrel,schoolsup,address,famsize,Mjob,Fjob,studytime
0,1,2,4,0,4,0,0,0,1,1,3,11,0,0,11,1,0,0,4,1,0,0,3,0,4,18,4,1,1,0,0,4,2
1,1,1,1,1,1,0,0,0,0,1,3,11,0,0,11,1,1,0,2,0,9,0,3,1,3,17,5,0,1,0,0,2,2
2,2,1,1,1,1,0,2,0,1,3,3,13,0,0,12,1,0,0,6,1,12,0,3,1,2,15,4,1,1,1,0,2,2
3,1,1,4,1,2,0,1,0,1,1,2,14,1,1,14,1,1,0,0,1,14,0,5,1,2,15,3,0,1,0,1,3,3
4,1,1,4,1,3,0,3,1,1,2,4,12,1,0,13,1,1,0,6,1,12,0,5,1,2,16,5,0,1,1,3,2,2


### pass or fail(based on result*60%)

In [15]:
def result_to_pass_fail(data, total_mark, pass_percentage):
    pass_fail = {}
    if type(data) == pd.DataFrame:
        for i in range(len(data)):
            result_sum = sum(data.iloc[i])
            if result_sum >= total_mark*pass_percentage:
                pass_fail[i] = 1
            else:
                pass_fail[i] = 0
    else:
        for i in range(len(data)):
            if data[i] >=  total_mark*pass_percentage:
                pass_fail[i] = 1
            else:
                pass_fail[i] = 0
    return pass_fail

In [16]:
total_result = data_train_2[["G1", "G2", "G3"]]
por_pass_fail = result_to_pass_fail(total_result, 60, 0.6)

In [17]:
data_train_2['pass_fail'] = por_pass_fail.values()

### replace 0 with -1

In [18]:
def replace_num(df):
    binary_data = df[["school","sex","address","famsize","Pstatus","schoolsup","famsup","activities","nursery","higher","internet"]]
    binary_data = binary_data.replace(0,-1)
    df = df.drop(columns = ["school","sex","address","famsize","Pstatus","schoolsup","famsup","activities","nursery","higher","internet"])
    df = pd.concat([df,binary_data], axis = 1)
    return df

In [19]:
data_train_2 = replace_num(data_train_2)

### randomforest and its result

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate

In [21]:
x = data_train_2.drop(columns = ["G1", "G2", "G3", "pass_fail"])
y = data_train_2["pass_fail"]
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.8, random_state = 1039)
randomforest_clf = RandomForestClassifier(n_estimators = 300, max_depth = 5)
randomforest_clf.fit(train_x, train_y)
pred_y = randomforest_clf.predict(test_x)
print(classification_report(pred_y, test_y))
print(confusion_matrix(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.76      0.62      0.69        56
           1       0.66      0.78      0.71        51

    accuracy                           0.70       107
   macro avg       0.71      0.70      0.70       107
weighted avg       0.71      0.70      0.70       107

[[35 11]
 [21 40]]


In [22]:
random_forest_cv = cross_validate(randomforest_clf, x, y, cv=5, scoring = 'f1')
print(random_forest_cv['test_score'])
print(np.mean(random_forest_cv['test_score']))

[0.71428571 0.77966102 0.72072072 0.66666667 0.11111111]
0.5984890459466731


In [23]:
def feature_importance_choosing(feature_importance, column_name, gate):
    choosing_feature = {}
    for i in range(len(feature_importance)):
        if feature_importance[i] >= gate:
            choosing_feature[column_name[i]] = feature_importance[i]
    return choosing_feature

In [24]:
forest_importances = randomforest_clf.feature_importances_
feature_name = list(x.columns)
feature_choose = feature_importance_choosing(forest_importances, feature_name, 0.03)

In [25]:
x = data_train_2[feature_choose]
y = data_train_2["pass_fail"]
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.8, random_state = 1039)
randomforest_clf = RandomForestClassifier(n_estimators = 300, max_depth = 5)
randomforest_clf.fit(train_x, train_y)
pred_y = randomforest_clf.predict(test_x)
print(classification_report(pred_y, test_y))
confusion_matrix(test_y, pred_y)

              precision    recall  f1-score   support

           0       0.76      0.65      0.70        54
           1       0.69      0.79      0.74        53

    accuracy                           0.72       107
   macro avg       0.72      0.72      0.72       107
weighted avg       0.73      0.72      0.72       107



array([[35, 11],
       [19, 42]], dtype=int64)

In [28]:
random_forest_cv = cross_validate(randomforest_clf, x, y, cv=5, scoring = 'f1')
print(random_forest_cv['test_score'])
print(np.mean(random_forest_cv['test_score']))

[0.70422535 0.78333333 0.73043478 0.68421053 0.        ]
0.5804407988740989


### Gradient Boosting and its result

In [29]:
from sklearn.ensemble import GradientBoostingClassifier

In [31]:
x = data_train_2.drop(columns = ["G1", "G2", "G3", "pass_fail"])
y = data_train_2["pass_fail"]
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.8, random_state = 1039)
gradient_clf = GradientBoostingClassifier(learning_rate = 0.01, n_estimators = 1000, max_depth = 5, random_state = 1039)
gradient_clf.fit(train_x, train_y)
pred_y = gradient_clf.predict(test_x)
print(classification_report(pred_y, test_y))
print(confusion_matrix(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.76      0.62      0.69        56
           1       0.66      0.78      0.71        51

    accuracy                           0.70       107
   macro avg       0.71      0.70      0.70       107
weighted avg       0.71      0.70      0.70       107

[[35 11]
 [21 40]]


In [32]:
gradient_cv = cross_validate(gradient_clf, x, y, cv=5, scoring = 'f1')
print(gradient_cv['test_score'])
print(np.mean(gradient_cv['test_score']))

[0.67692308 0.73214286 0.61818182 0.66037736 0.20689655]
0.5789043324924912


In [33]:
gradient_importances = gradient_clf.feature_importances_
feature_name = list(x.columns)
feature_choose_gradient = feature_importance_choosing(gradient_importances, feature_name, 0.03)

In [34]:
x = data_train_2[feature_choose_gradient]
y = data_train_2["pass_fail"]
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.8, random_state = 1039)
gradient_clf = GradientBoostingClassifier(learning_rate = 0.01, n_estimators = 1000, max_depth = 5, random_state = 1039)
gradient_clf.fit(train_x, train_y)
pred_y = gradient_clf.predict(test_x)
print(classification_report(pred_y, test_y))
print(confusion_matrix(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.80      0.62      0.70        60
           1       0.62      0.81      0.70        47

    accuracy                           0.70       107
   macro avg       0.71      0.71      0.70       107
weighted avg       0.72      0.70      0.70       107

[[37  9]
 [23 38]]


In [35]:
gradient_cv = cross_validate(gradient_clf, x, y, cv=5, scoring = 'f1')
print(gradient_cv['test_score'])
print(np.mean(gradient_cv['test_score']))

[0.66666667 0.69158879 0.7079646  0.69026549 0.1754386 ]
0.5863848273400398


### XGBoosting and its result

In [36]:
import xgboost
from xgboost.sklearn import XGBClassifier

c:\users\yang\appdata\local\programs\python\python35\lib\site-packages\xgboost\__init__.py:29: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)


In [37]:
x = data_train_2.drop(columns = ["G1", "G2", "G3", "pass_fail"])
y = data_train_2["pass_fail"]
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.8, random_state = 1039)
xgb_clf = XGBClassifier(learning_rate = 0.01, alpha = 0.1, max_depth = 8)
xgb_clf.fit(train_x, train_y)
pred_y = xgb_clf.predict(test_x)
print(classification_report(pred_y, test_y))
print(confusion_matrix(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.67      0.63      0.65        49
           1       0.70      0.74      0.72        58

    accuracy                           0.69       107
   macro avg       0.69      0.69      0.69       107
weighted avg       0.69      0.69      0.69       107

[[31 15]
 [18 43]]


In [38]:
xgb_cv = cross_validate(xgb_clf, x, y, cv=5, scoring = 'f1')
print(xgb_cv['test_score'])
print(np.mean(xgb_cv['test_score']))

[0.69565217 0.7027027  0.72072072 0.72       0.13793103]
0.595401326363845


In [40]:
xgb_importances = xgb_clf.feature_importances_
feature_name = list(x.columns)
feature_choose_xgb = feature_importance_choosing(xgb_importances, feature_name, 0.03)

In [41]:
x = data_train_2[feature_choose_xgb]
y = data_train_2["pass_fail"]
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.8, random_state = 1039)
xgb_clf = XGBClassifier(learning_rate = 0.01, alpha = 0.1, max_depth = 8)
xgb_clf.fit(train_x, train_y)
pred_y = xgb_clf.predict(test_x)
print(classification_report(pred_y, test_y))
print(confusion_matrix(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.57      0.60      0.58        43
           1       0.72      0.69      0.70        64

    accuracy                           0.65       107
   macro avg       0.64      0.65      0.64       107
weighted avg       0.66      0.65      0.66       107

[[26 20]
 [17 44]]


In [42]:
xgb_cv = cross_validate(xgb_clf, x, y, cv=5, scoring = 'f1')
print(xgb_cv['test_score'])
print(np.mean(xgb_cv['test_score']))

[0.68571429 0.77586207 0.77310924 0.68421053 0.        ]
0.5837792249386142


### model feature choose compare

In [43]:
forest_feature = pd.DataFrame(feature_choose.values(), index = feature_choose.keys(), columns = ['RandomForest_clf'])
gradient_feature = pd.DataFrame(feature_choose_gradient.values(), index = feature_choose_gradient.keys(), columns = ['GradientBoosting_clf'])
XGB_feature = pd.DataFrame(feature_choose_xgb.values(), index = feature_choose_xgb.keys(), columns = ['XGBoosting_clf'])

In [44]:
feature_choose_compare = pd.concat([forest_feature,gradient_feature,XGB_feature],axis = 1)
feature_choose_compare = feature_choose_compare.fillna(0)
feature_choose_compare

c:\users\yang\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,RandomForest_clf,GradientBoosting_clf,XGBoosting_clf
Dalc,0.037187,0.031865,0.000000
Fedu,0.038267,0.037250,0.000000
Medu,0.044315,0.000000,0.000000
Mjob,0.033608,0.042173,0.000000
Walc,0.050030,0.042928,0.038061
absences,0.077664,0.098264,0.036358
address,0.042519,0.000000,0.000000
age,0.030596,0.039930,0.000000
failures,0.168967,0.155024,0.269781
famrel,0.000000,0.043363,0.000000


### Test_result

In [45]:
def data_preprocessing(df):
    df, a = categorical_transform(df)
    test_result = df[["G1", "G2", "G3"]]
    test_pass_fail = result_to_pass_fail(test_result, 60, 0.6)
    df['pass_fail'] = test_pass_fail.values()
    df = replace_num(df)
    return df

In [46]:
data_test_2 = data_preprocessing(data_test)

In [48]:
x = data_train_2[feature_choose]
y = data_train_2["pass_fail"]
test_x = data_test_2[feature_choose]
test_y = data_test_2["pass_fail"]
randomforest_clf = RandomForestClassifier(n_estimators = 300, max_depth = 5)
randomforest_clf.fit(x, y)
pred_y = randomforest_clf.predict(test_x)
print(classification_report(pred_y, test_y))
confusion_matrix(test_y, pred_y)

              precision    recall  f1-score   support

           0       0.69      0.73      0.70        51
           1       0.70      0.65      0.67        49

    accuracy                           0.69       100
   macro avg       0.69      0.69      0.69       100
weighted avg       0.69      0.69      0.69       100



array([[37, 17],
       [14, 32]], dtype=int64)

In [49]:
x = data_train_2[feature_choose_gradient]
y = data_train_2["pass_fail"]
test_x = data_test_2[feature_choose_gradient]
test_y = data_test_2["pass_fail"]
gradient_clf = GradientBoostingClassifier(learning_rate = 0.01, n_estimators = 1000, max_depth = 5, random_state = 1039)
gradient_clf.fit(x, y)
pred_y = gradient_clf.predict(test_x)
print(classification_report(pred_y, test_y))
print(confusion_matrix(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.67      0.68      0.67        53
           1       0.63      0.62      0.62        47

    accuracy                           0.65       100
   macro avg       0.65      0.65      0.65       100
weighted avg       0.65      0.65      0.65       100

[[36 18]
 [17 29]]


In [50]:
x = data_train_2[feature_choose_xgb]
y = data_train_2["pass_fail"]
test_x = data_test_2[feature_choose_xgb]
test_y = data_test_2["pass_fail"]
xgb_clf = XGBClassifier(learning_rate = 0.01, alpha = 0.1, max_depth = 5)
xgb_clf.fit(x, y)
pred_y = xgb_clf.predict(test_x)
print(classification_report(pred_y, test_y))
print(confusion_matrix(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.67      0.72      0.69        50
           1       0.70      0.64      0.67        50

    accuracy                           0.68       100
   macro avg       0.68      0.68      0.68       100
weighted avg       0.68      0.68      0.68       100

[[36 18]
 [14 32]]


# Regression Part

In [51]:
rd.seed(8719)
data_dict_2 = data_por_2.to_dict("index")
data_dict_list_2 = list(data_dict_2)
test_index_2 = rd.sample(data_dict_list_2, k = 100)
data_reg_train, data_reg_test = train_test_seperate(data_dict_2, data_dict_list_2, test_index_2)

In [52]:
data_train_reg_2 = data_preprocessing(data_reg_train)
data_test_reg_2 = data_preprocessing(data_reg_test)

### SVM regression model

In [54]:
#from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import math
from sklearn.metrics import r2_score

### gradient boosting regression model

In [55]:
from sklearn.ensemble import GradientBoostingRegressor

In [56]:
x = data_train_reg_2.drop(columns = ["G1", "G2", "G3", "pass_fail"])
y = data_train_reg_2["G3"]
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.8, random_state = 1039)
grad_regression = GradientBoostingRegressor(learning_rate = 0.01, n_estimators = 1000, max_depth = 5)
grad_regression.fit(train_x, train_y)
grad_pred_y = grad_regression.predict(test_x)
print(mean_squared_error(test_y, grad_pred_y))
print(math.sqrt(mean_squared_error(test_y, grad_pred_y)))
print(r2_score(test_y, grad_pred_y))

6.5312579705167
2.5556325969349936
0.14249245998433857


In [57]:
cv_results = cross_validate(grad_regression, x, y, cv=5, scoring = 'neg_mean_squared_error')
print(-cv_results['test_score'])
print(-np.mean(cv_results['test_score']))

[4.06935035 5.56948829 5.90695478 5.22966743 7.9990195 ]
5.754896072183358


In [59]:
gradient_importances_2 = grad_regression.feature_importances_
feature_name = list(x.columns)
feature_choose_gradient_2 = feature_importance_choosing(gradient_importances_2, feature_name, 0.03)

In [60]:
x = data_train_reg_2[feature_choose_gradient_2]
y = data_train_reg_2["G3"]
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.8, random_state = 1039)
grad_regression = GradientBoostingRegressor(learning_rate = 0.01, n_estimators = 1000, max_depth =5)
grad_regression.fit(train_x, train_y)
grad_pred_y = grad_regression.predict(test_x)
print(mean_squared_error(test_y, grad_pred_y))
print(math.sqrt(mean_squared_error(test_y, grad_pred_y)))
print(r2_score(test_y, grad_pred_y))

6.9040238687467275
2.627550926004428
0.0935509590000081


In [61]:
cv_results = cross_validate(grad_regression, x, y, cv=5, scoring = 'neg_mean_squared_error')
print(-cv_results['test_score'])
print(-np.mean(cv_results['test_score']))

[4.94363687 6.14622619 7.00426041 4.85226474 7.91381068]
6.172039778677613


### random forest regression

In [62]:
from sklearn.ensemble import RandomForestRegressor

In [63]:
x = data_train_reg_2.drop(columns = ["G1", "G2", "G3", "pass_fail"])
y = data_train_reg_2["G3"]
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.8, random_state = 1039)
random_regression = RandomForestRegressor(max_depth = 5, n_estimators = 300)
random_regression.fit(train_x, train_y)
random_pred_y = random_regression.predict(test_x)
print(mean_squared_error(test_y, random_pred_y))
print(math.sqrt(mean_squared_error(test_y, random_pred_y)))
print(r2_score(test_y, random_pred_y))

6.051263427965643
2.459931590098725
0.20551231638289658


In [64]:
cv_results = cross_validate(random_regression, x, y, cv=5, scoring = 'neg_mean_squared_error')
print(-cv_results['test_score'])
print(-np.mean(cv_results['test_score']))

[3.9290665  5.03658881 6.10572448 4.05238272 7.60959109]
5.346670718623425


In [66]:
forest_importances_2 = random_regression.feature_importances_
feature_name = list(x.columns)
feature_choose_forest_2 = feature_importance_choosing(forest_importances_2, feature_name, 0.03)

In [67]:
x = data_train_reg_2[feature_choose_forest_2]
y = data_train_reg_2["G3"]
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.8, random_state = 1039)
random_regression = RandomForestRegressor(max_depth = 5, n_estimators = 300)
random_regression.fit(train_x, train_y)
random_pred_y = random_regression.predict(test_x)
print(mean_squared_error(test_y, random_pred_y))
print(math.sqrt(mean_squared_error(test_y, random_pred_y)))
print(r2_score(test_y, random_pred_y))

6.23468389983181
2.496934900999986
0.18143051800217413


In [68]:
cv_results = cross_validate(random_regression, x, y, cv=5, scoring = 'neg_mean_squared_error')
print(-cv_results['test_score'])
print(-np.mean(cv_results['test_score']))

[3.76933244 5.11465382 6.25858068 4.29748698 7.21392094]
5.330794971792481


### XGBRegressor and its result

In [69]:
from xgboost import XGBRegressor

In [70]:
x = data_train_reg_2.drop(columns = ["G1", "G2", "G3", "pass_fail"])
y = data_train_reg_2["G3"]
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.8, random_state = 1039)
xgb_regression = XGBRegressor(learning_rate = 0.01, alpha = 0.1, max_depth = 5)
xgb_regression.fit(train_x, train_y)
xgb_pred_y = xgb_regression.predict(test_x)
print(mean_squared_error(test_y, xgb_pred_y))
print(math.sqrt(mean_squared_error(test_y, xgb_pred_y)))
print(r2_score(test_y, xgb_pred_y))

23.486796795932808
4.846317859564394
-2.083648729577702


In [71]:
cv_results = cross_validate(xgb_regression, x, y, cv=5, scoring = 'neg_mean_squared_error')
print(-cv_results['test_score'])
print(-np.mean(cv_results['test_score']))

[22.20885412 20.66711204 31.59990236 26.6074362  22.81065121]
24.778791183872354


In [72]:
xgb_importances_2 = xgb_regression.feature_importances_
feature_name = list(x.columns)
feature_choose_xgb_2 = feature_importance_choosing(xgb_importances_2, feature_name, 0.03)

In [73]:
x = data_train_reg_2[feature_choose_xgb_2]
y = data_train_reg_2["G3"]
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.8, random_state = 1039)
xgb_regression = XGBRegressor(learning_rate = 0.01, alpha = 0.1, max_depth = 5)
xgb_regression.fit(train_x, train_y)
xgb_pred_y = xgb_regression.predict(test_x)
print(mean_squared_error(test_y, xgb_pred_y))
print(math.sqrt(mean_squared_error(test_y, xgb_pred_y)))
print(r2_score(test_y, xgb_pred_y))

23.578732276950635
4.855793681464507
-2.0957192018394974


In [74]:
cv_results = cross_validate(xgb_regression, x, y, cv=5, scoring = 'neg_mean_squared_error')
print(-cv_results['test_score'])
print(-np.mean(cv_results['test_score']))

[19.89012429 21.98327498 33.2871599  25.82920063 21.2226296 ]
24.44247787990366


### Feature Compare

In [75]:
forest_feature_2 = pd.DataFrame(random_regression.feature_importances_, index = feature_choose_forest_2.keys(), columns = ['RandomForest_reg'])
gradient_feature_2 = pd.DataFrame(grad_regression.feature_importances_, index = feature_choose_gradient_2.keys(), columns = ['GradientBoosting_reg'])
XGB_feature_2 = pd.DataFrame(xgb_regression.feature_importances_, index = feature_choose_xgb_2.keys(), columns = ['XGBoosting_reg'])

In [76]:
feature_choose_compare_2 = pd.concat([forest_feature_2,gradient_feature_2,XGB_feature_2],axis = 1)
feature_choose_compare_2 = feature_choose_compare_2.fillna(0)
feature_choose_compare_2

c:\users\yang\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,RandomForest_reg,GradientBoosting_reg,XGBoosting_reg
Dalc,0.057245,0.056095,0.000000
Fedu,0.000000,0.061460,0.000000
Fjob,0.000000,0.052211,0.000000
Medu,0.106456,0.099415,0.042016
Walc,0.055942,0.051430,0.026803
absences,0.090067,0.101589,0.000000
address,0.000000,0.000000,0.021665
age,0.126148,0.084521,0.000000
failures,0.322562,0.202105,0.640623
goout,0.000000,0.065369,0.000000


### Regressor and Classifier compare

In [77]:
pd.concat([feature_choose_compare_2,feature_choose_compare], axis = 1)

c:\users\yang\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,RandomForest_reg,GradientBoosting_reg,XGBoosting_reg,RandomForest_clf,GradientBoosting_clf,XGBoosting_clf
Dalc,0.057245,0.056095,0.000000,0.037187,0.031865,0.000000
Fedu,0.000000,0.061460,0.000000,0.038267,0.037250,0.000000
Fjob,0.000000,0.052211,0.000000,NaN,NaN,NaN
Medu,0.106456,0.099415,0.042016,0.044315,0.000000,0.000000
Mjob,NaN,NaN,NaN,0.033608,0.042173,0.000000
Walc,0.055942,0.051430,0.026803,0.050030,0.042928,0.038061
absences,0.090067,0.101589,0.000000,0.077664,0.098264,0.036358
address,0.000000,0.000000,0.021665,0.042519,0.000000,0.000000
age,0.126148,0.084521,0.000000,0.030596,0.039930,0.000000
failures,0.322562,0.202105,0.640623,0.168967,0.155024,0.269781


In [78]:
x = data_train_reg_2[feature_choose_forest_2]
y = data_train_reg_2["G3"]
test_x = data_test_reg_2[feature_choose_forest_2]
test_y = data_test_reg_2["G3"]
random_regression = RandomForestRegressor(max_depth = 5, n_estimators = 300)
random_regression.fit(x, y)
random_pred_y = random_regression.predict(test_x)
print(mean_squared_error(test_y, random_pred_y))
print(math.sqrt(mean_squared_error(test_y, random_pred_y)))
print(r2_score(test_y, random_pred_y))

4.36256564085685
2.0886755709915437
0.2300313029074198


In [79]:
x = data_train_reg_2[feature_choose_gradient_2]
y = data_train_reg_2["G3"]
test_x = data_test_reg_2[feature_choose_gradient_2]
test_y = data_test_reg_2["G3"]
grad_regression = GradientBoostingRegressor(learning_rate = 0.01, n_estimators = 1000, max_depth =5)
grad_regression.fit(x, y)
grad_pred_y = grad_regression.predict(test_x)
print(mean_squared_error(test_y, grad_pred_y))
print(math.sqrt(mean_squared_error(test_y, grad_pred_y)))
print(r2_score(test_y, grad_pred_y))

5.142273715904077
2.2676582008548105
0.09241714186553307


In [80]:
x = data_train_reg_2[feature_choose_xgb_2]
y = data_train_reg_2["G3"]
test_x = data_test_reg_2[feature_choose_xgb_2]
test_y = data_test_reg_2["G3"]
xgb_regression = XGBRegressor(learning_rate = 0.01, alpha = 0.1, max_depth = 5)
xgb_regression.fit(x, y)
xgb_pred_y = xgb_regression.predict(test_x)
print(mean_squared_error(test_y, xgb_pred_y))
print(math.sqrt(mean_squared_error(test_y, xgb_pred_y)))
print(r2_score(test_y, xgb_pred_y))

23.18094865432652
4.81465976516789
-3.091309174945996
